In [3]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.datasets import MNIST
from torch.autograd import Variable

In [6]:
# CapsNet
from capsnet import CapsNetWithReconstruction, CapsNet, ReconstructionNet

# standard ConvNet
from convnet import Net

usage: ipykernel_launcher.py [-h] [--batch-size N] [--test-batch-size N]
                             [--epochs N] [--lr LR] [--momentum M] [--no-cuda]
                             [--seed S] [--log-interval N]
ipykernel_launcher.py: error: unrecognized arguments: -f /run/user/1000/jupyter/kernel-28b8ca87-45ea-4d88-bc87-dd2bab4b17fa.json


SystemExit: 2